In [72]:
import pandas as pd
import numpy as np
from ast import literal_eval

## Orders

In [4]:
orders_df = pd.read_csv('data/train_orders.csv', index_col=0)

In [34]:
orders_df.head()

,user-id,site-id,created-at,item-id,count,general-category-path,brand-id,target
0,user_1,1,1634292444,NaN,NaN,NaN,NaN,0
1,user_1,2,1639827297,NaN,NaN,NaN,NaN,0
2,user_1,2,1639827681,NaN,NaN,NaN,NaN,0
3,user_3,21,1684248081,NaN,NaN,NaN,NaN,1
4,user_3,22,1684892551,item_1,1.0,NaN,NaN,1


Константы

In [100]:
# Граница отсечения результатов (настраивается в процессе тестов)
BORDER = 100

### Группировки

Разобьем список категория на отдельные строки

In [101]:
orders_df_with_category = orders_df[~orders_df['general-category-path'].isna()]
orders_df_with_category['general-category-path'] = orders_df_with_category['general-category-path'].apply(literal_eval)
orders_df_with_category = orders_df_with_category.explode('general-category-path')
orders_df_with_category

C:\Users\Demion\AppData\Local\Temp\ipykernel_31776\18327562.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  orders_df_with_category['general-category-path'] = orders_df_with_category['general-category-path'].apply(literal_eval)


,user-id,site-id,created-at,item-id,count,general-category-path,brand-id,target
6,user_4,29,1611987662,item_9,1.0,90404,3243.0,1
6,user_4,29,1611987662,item_9,1.0,90403,3243.0,1
6,user_4,29,1611987662,item_9,1.0,90402,3243.0,1
7,user_4,29,1655426430,item_10,1.0,90698,1210537.0,1
7,user_4,29,1655426430,item_10,1.0,7330336,1210537.0,1
...,...,...,...,...,...,...,...,...
3458394,user_127755,58,1690265390,item_1896437,1.0,90813,14019.0,0
3458395,user_127755,58,1690265390,item_3739266,1.0,12718223,32654.0,0
3458395,user_127755,58,1690265390,item_3739266,1.0,90813,32654.0,0
3458396,user_127755,58,1690265390,item_57560,1.0,12718223,5783.0,0


#### Сайт/категория

In [86]:
orders_df_with_category.groupby(by=['site-id', 'general-category-path']).agg(
    Count=('created-at', 'nunique'),
    Gender=('target', np.mean)
).query(f'Count >= {BORDER}').sort_values(by='Gender', ascending=False)

Count    Gender
site-id general-category-path                 
20      7774364                  160  0.931174
        91514                    239  0.914948
        90403                    109  0.914530
151     12333552                 143  0.908108
25      90402                    160  0.906593
...                              ...       ...
146     91325                    130  0.024793
588     13238924                 216  0.024145
332     7811873                  101  0.021505
136     8475933                  187  0.020690
108     91182                    157  0.017668

[4761 rows x 2 columns]

#### Бренд

In [99]:
orders_df.groupby(by=['brand-id'])['user-id', 'created-at', 'target'].agg(
    Count=('created-at', 'count'),
    Gender=('target', np.mean)
).query(f'Count >= {BORDER}').sort_values(by='Gender', ascending=False)

C:\Users\Demion\AppData\Local\Temp\ipykernel_31776\1772745303.py:1: FutureWarning: Indexing with multiple keys (implicitly converted to a tuple of keys) will be deprecated, use a list instead.
  orders_df.groupby(by=['brand-id'])['user-id', 'created-at', 'target'].agg(


,Count,Gender
brand-id,,
1002.0,122,0.942623
993.0,107,0.878505
10418.0,169,0.875740
964.0,110,0.872727
5877.0,122,0.868852
...,...,...
80069.0,163,0.012270
33350.0,147,0.006803
24022.0,228,0.000000


#### Сайт

In [95]:
orders_df.groupby(by=['site-id'])['user-id', 'created-at', 'target'].agg(
    Count=('created-at', 'count'),
    Gender=('target', np.mean)
).query(f'Count >= {BORDER}').sort_values(by='Gender', ascending=False)

C:\Users\Demion\AppData\Local\Temp\ipykernel_31776\1879741261.py:1: FutureWarning: Indexing with multiple keys (implicitly converted to a tuple of keys) will be deprecated, use a list instead.
  orders_df.groupby(by=['site-id'])['user-id', 'created-at', 'target'].agg(


,Count,Gender
site-id,,
599,254,0.889764
830,399,0.864662
139,375,0.858667
488,12358,0.849086
848,112,0.812500
...,...,...
1264,132,0.030303
931,142,0.028169
717,168,0.017857


#### Сайт/бренд

In [88]:
orders_df.groupby(by=['site-id', 'brand-id']).agg(
    Count=('created-at', 'nunique'),
    Gender=('target', np.mean)
).query(f'Count >= {BORDER}').sort_values(by='Gender', ascending=False)

Count    Gender
site-id brand-id                  
20      1211265.0    104  0.908257
151     10418.0      129  0.888889
        1946.0       248  0.875000
20      12052.0      123  0.862595
        1211.0       128  0.861538
...                  ...       ...
45      12655.0      270  0.038540
149     953508.0     179  0.037815
        12677.0      193  0.037736
199     1940278.0    102  0.018634
        5430.0       126  0.016129

[1588 rows x 2 columns]

#### Сайт/товар

In [52]:
orders_df.groupby(by=['site-id', 'item-id']).agg(
    Count=('created-at', 'nunique'),
    Gender=('target', np.mean)
).query(f'Count >= {BORDER}').sort_values(by='Gender', ascending=False)

Count    Gender
site-id item-id                    
151     item_23902    230  0.860870
        item_3327     399  0.856079
352     item_42908    125  0.840237
20      item_7298     100  0.811881
151     item_2563     233  0.807531
...                   ...       ...
104     item_27114    107  0.036697
149     item_25819    134  0.036232
146     item_7357     115  0.031250
136     item_18595    128  0.031008
439     item_57660    117  0.024793

[937 rows x 2 columns]

#### Конвертация в словарь для поиска при оценке

In [30]:
orders_site_item_count.to_dict()

{(54, 'item_63'): 22238,
 (2, 'item_8'): 20872,
 (2, 'item_424'): 17068,
 (184, 'item_6631'): 14171,
 (39, 'item_429'): 13903,
 (2, 'item_1519'): 10974,
 (179, 'item_210'): 9303,
 (93, 'item_7606'): 7360,
 (194, 'item_4305'): 6892,
 (653, 'item_2782'): 5021,
 (441, 'item_680'): 4959,
 (645, 'item_2346'): 4604,
 (34, 'item_1781'): 4329,
 (49, 'item_1521'): 4293,
 (93, 'item_3004'): 3872,
 (93, 'item_3005'): 3862,
 (93, 'item_3006'): 3849,
 (56, 'item_135'): 3457,
 (57, 'item_14051'): 3272,
 (650, 'item_2624'): 3103,
 (439, 'item_433'): 2807,
 (58, 'item_5507'): 2663,
 (86, 'item_860'): 2660,
 (193, 'item_1970'): 2655,
 (346, 'item_7499'): 2593,
 (93, 'item_4930'): 2566,
 (8, 'item_1349'): 2514,
 (16, 'item_2002'): 2498,
 (182, 'item_2290'): 2415,
 (6, 'item_936'): 2303,
 (590, 'item_8191'): 2197,
 (34, 'item_1648'): 2147,
 (6, 'item_2654'): 2109,
 (6, 'item_809'): 2105,
 (341, 'item_4567'): 2050,
 (181, 'item_5336'): 2047,
 (151, 'item_3826'): 1997,
 (40, 'item_4241'): 1992,
 (44, 'item

## Visits (позже)

In [10]:
visits_df = pd.read_csv('data/train_visits.csv', index_col=0)

In [12]:
visits_df.head(50)

,user-id,site-id,first-seen,last-seen,visited-at,session-duration,pages-count,visited-items,visited-general-categories,visited-universal_brands,target
0,user_2,3,1696277805,1696278323,1696277805,0,1,NaN,NaN,NaN,0
1,user_2,3,1696277805,1696278323,1696278323,112,2,NaN,NaN,NaN,0
2,user_3,3,1696320204,1696320204,1696320204,0,1,NaN,NaN,NaN,1
3,user_3,23,1686372887,1686899354,1686372887,296,13,['item_2'],NaN,NaN,1
4,user_3,23,1686372887,1686899354,1686805600,37,5,['item_2'],NaN,NaN,1
5,user_3,23,1686372887,1686899354,1686899354,99,5,NaN,NaN,NaN,1
6,user_3,16,1680410306,1686982782,1681145345,558,6,"['item_3', 'item_4', 'item_5']","['91461', '198119', '91491']","['1', '921']",1
7,user_3,16,1680410306,1686982782,1686373121,93,3,NaN,NaN,NaN,1
8,user_3,16,1680410306,1686982782,1686711421,1189,10,"['item_6', 'item_7']","['91461', '198119', '91491']",['1'],1
9,user_3,16,1680410306,1686982782,1686805443,105,2,NaN,NaN,NaN,1
